# Hotel recommendation from traveller request in natural language

AI Agent that recommends a hotel from traveller preferences given in natural language.

## 1. Define agent with LangGraph

In [ ]:
!pip install --upgrade --quiet langchain
!pip install --upgrade --quiet langchain-community
!pip install --upgrade --quiet langgraph

In [ ]:
from langchain_core.prompts import PromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict, Dict
import json

class AgentState(TypedDict):
    traveller_request: str
    hotel: Dict
    room: Dict

class TravelAgent:
    def __init__(self, model, vector_store):
        self.model = model
        self.vector_store = vector_store
        self.prompt_travel_agent = "You are a travel agent that helps travellers to book hotels"
        
        workflow = StateGraph(AgentState)
        workflow.add_node("extract_hotel", self.extract_hotel)
        workflow.add_node("search_hotel", self.search_hotel)
        workflow.add_node("extract_room", self.extract_room)
        workflow.set_entry_point("extract_hotel")
        workflow.add_conditional_edges(
            "extract_hotel",
            self.should_extract_room,
            {"room": "extract_room", "search": "search_hotel"}
        )
        workflow.add_edge("search_hotel", "extract_room")
        workflow.add_edge("extract_room", END)
        self.graph = workflow.compile()
        
    def extract_hotel(self, state: AgentState):
        prompt_extract_hotel_destination_template = PromptTemplate.from_template("""
            You are required by a traveller to book a hotel. From the text delimited by triple backticks extract the hotel name and the destination the traveller wants to stay.
            ```
            {traveller_request}
            ```

            Return your response as a json object using the following schema:
            {{
            hotel: "Hotel name",
            destination: "Destination of the stay"
            }}

            Do not be verbose and do not reason your response.
            Just return a json object.
            Do not delimit response in triple backtips.
            Do not invent the response if you do not know it return empty value.
        """)
        messages = [
            {"role": "system", "content": self.prompt_travel_agent},
            {"role": "user", "content": prompt_extract_hotel_destination_template.format(traveller_request=state['traveller_request'])}
        ]
        output = self.model.invoke(messages)
        
        hotel_destination = json.loads(output.content)
        hotel_name = hotel_destination['hotel']
        if hotel_name:
            # Search by hotel name in the database
            hotels = vector_store.similarity_search('', k=1, filter={'name': hotel_name})
            return {'hotel': hotels[0].metadata if len(hotels)>0 else None}
        else:
            return {'hotel': None}
        
    def should_extract_room(self, state: AgentState):
        if bool(state['hotel']):
            return("room")
        else:
            return("search")
        
    def search_hotel(self, state: AgentState):
        prompt_build_hotel_recommendation = PromptTemplate.from_template("""
        You are required by a traveller to book a hotel. From the text delimited by triple backticks describe a hotel that matches the needings of the traveller.
        Do not recommend an specific hotel.
        Do not be verbose, just write the description of the recommended hotel. You are not a chatbot.
        ```
        {traveller_request}
        ```
        Do not delimit response in triple backtips.
        """)
        messages = [
            {"role": "system", "content": self.prompt_travel_agent},
            {"role": "user", "content": prompt_build_hotel_recommendation.format(traveller_request=state['traveller_request'])}
        ]
        hotel_recommendation = self.model.invoke(messages)
        hotels = vector_store.similarity_search(hotel_recommendation, k=1)
        
        return {'hotel': hotels[0].metadata}
    
    def extract_room(self, state: AgentState):
        prompt_build_room_recommendation = PromptTemplate.from_template("""
        You are required by a traveller to book a hotel. From the text delimited by triple backticks describe a room that matches the needings and requirements of the traveller.
        Do not be verbose. You are not a chatbot.
        Just write the description of the room.
        ```
        {traveller_request}
        ```
        Do not delimit response in triple backtips.
        """)
        messages = [
            {"role": "system", "content": self.prompt_travel_agent},
            {"role": "user", "content": prompt_build_room_recommendation.format(traveller_request=state['traveller_request'])}
        ]
        room_recommendation = self.model.invoke(messages)

        filter = {
            "$and":[
                {"type": "room"},
                {"hotel": state['hotel']['id']}
            ]
        }
        rooms = vector_store.similarity_search(room_recommendation, k=1, filter=filter)
        room = rooms[0].metadata
        
        return {'room': rooms[0].metadata}

    def invoke(self, request: str):
        return self.graph.invoke({
            'traveller_request': request,
            'hotel': None,
            'room': None
        })

In [ ]:
from helper import load_hotels
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import ChatOllama

vector_store = Chroma(embedding_function=OllamaEmbeddings(model="mxbai-embed-large"))
load_hotels(vector_store=vector_store)

model = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
)

travel_agent = TravelAgent(model=model, vector_store=vector_store)

In [ ]:
from IPython.display import Image, display
display(Image(travel_agent.graph.get_graph().draw_mermaid_png()))

In [ ]:
request = 'I want to visit Palma of Majorca and I want to stay near the beach. I am travelling alone and I want a quiet room.'

result = travel_agent.invoke(request)

print("Dear traveller!")
print(f"  Hotel: {result['hotel']['name']}")
print(f"  Room: {result['room']['name']}")